In [2]:
import numpy as np
import cv2
import cv2.aruco as aruco
import pathlib
import glob
import matplotlib.pyplot as plt



def draw_cube(img, imgpts):
    imgpts = np.int32(imgpts).reshape(-1, 2)
    print(imgpts)

    # draw floor
    floor = cv2.drawContours(img.copy(), [imgpts[:4]], -1, (255, 0, 0), -2)
    img = cv2.addWeighted(floor, 0.5, img, 0.9, 0)
    aruco.drawDetectedMarkers(img, corners)
    aruco.drawAxis(img, mtx, dist, rvecs, tvecs, 1)
    # draw pillars
    for i, j in zip(range(4), range(4, 8)):
        cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (0, 255, 0), 3)

    # draw roof
    roof = cv2.drawContours(img.copy(), [imgpts[4:]], -1, (0, 0, 255), -2)
    img = cv2.addWeighted(roof, 0.5, img, 0.9, 0)
#     cv2.imshow("winname", img)
#     cv2.waitKey(0)
    plt.figure(figsize=(40, 40))

    plt.imshow(img)



mtx = np.array(mtx)
dist = np.array(dist)
image = cv2.imread("/home/akanksha/Downloads/HW4/img1.jpg")
# image = cv2.imread("1.jpg")
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_1000)
arucoParams = aruco.DetectorParameters_create()
# load calibration data

# p = 1.15
# q = 2.3


p = 1.5
q = 2.3
axis = np.float32([[-p, -p, 0], [-p, p, 0], [p, p, 0], [p, -p, 0],
                   [-p, -p, q], [-p, p, q], [p, p, q], [p, -p, q]])
# find grid corners in image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("winname", gray)
# cv2.waitKey()
# plt.imshow(gray)
corners, ids, rejectedImgPoints = aruco.detectMarkers(
    gray, aruco_dict, parameters=arucoParams, cameraMatrix=mtx, distCoeff=dist)
# print(corners[0][0])
rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, 2.3, mtx, dist)

imgpts, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, distCoeffs=dist)

NameError: name 'mtx' is not defined